In [94]:
%load_ext autoreload
%autoreload 2
import unittest 
import sys
import os

sys.path.append(os.getcwd() + '/..')

from backend.AppApi import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
import pandas as pd
from Constants import Const
import matplotlib.pyplot as plt

In [3]:
#this loads in the processed dataset, does some cleaning, adds in some dose thresholds, and parses the string lists into actual lists
data = load_dose_symptom_data('minimal_dose_cluster_dataset.csv')
data.head()

,id,D10,D15,D2,D20,D25,D30,D35,D40,D45,...,total_mean_dose,Parotid_Gland_limit,IPC_limit,IPC_limit2,MPC_limit,MPC_limit2,SPC_limit,SPC_limit2,Larynx_limit,Esophagus_limit
0,2,"[51.625, 26.890625, 56.1875, 53.625, 53.75, 57...","[49.78125, 26.515625, 54.6875, 52.21875, 52.96...","[54.53125, 28.1875, 59.84375, 57.40625, 55.937...","[47.09375, 25.875, 53.09375, 50.875, 52.4375, ...","[44.4375, 25.453125, 51.375, 49.75, 52.03125, ...","[42.53125, 25.078125, 49.71875, 48.6875, 51.65...","[41.03125, 24.703125, 48.03125, 47.75, 51.3125...","[39.78125, 24.375, 46.46875, 46.625, 51.0, 51....","[38.4375, 24.125, 44.875, 45.6875, 50.5625, 51...",...,1733.224947,True,False,False,True,True,True,True,False,True
1,6,"[44.96875, 26.40625, 61.71875, 56.78125, 55.18...","[44.03125, 25.84375, 58.75, 55.9375, 52.40625,...","[48.0625, 28.03125, 65.5625, 58.3125, 61.28125...","[42.78125, 25.40625, 56.71875, 54.875, 50.4687...","[41.4375, 25.03125, 55.46875, 53.625, 49.5625,...","[40.375, 24.65625, 53.4375, 52.375, 49.0, 50.0...","[39.59375, 24.3125, 51.65625, 51.125, 47.8125,...","[39.09375, 24.0, 50.09375, 49.84375, 46.5625, ...","[38.625, 23.671875, 48.53125, 48.75, 45.59375,...",...,1827.967819,True,False,False,True,False,True,True,False,True
2,7,"[37.625, 37.15625, 66.3125, 60.3125, 49.4375, ...","[36.5625, 36.75, 64.0625, 59.96875, 48.375, 49...","[39.96875, 38.0625, 71.8125, 60.90625, 52.1562...","[35.25, 36.34375, 62.71875, 59.71875, 47.34375...","[34.0625, 35.90625, 61.53125, 59.40625, 46.562...","[33.125, 35.5, 60.34375, 59.1875, 45.6875, 43....","[32.28125, 35.03125, 59.03125, 58.90625, 44.75...","[31.609375, 34.625, 57.71875, 58.625, 43.96875...","[30.84375, 34.1875, 56.40625, 58.3125, 43.1875...",...,1795.847534,True,True,False,True,False,True,False,False,False
3,8,"[32.15625, 32.0625, 65.375, 56.6875, 22.5, 31....","[31.703125, 31.109375, 63.34375, 55.75, 21.312...","[33.4375, 34.09375, 68.75, 58.28125, 33.6875, ...","[31.203125, 30.25, 61.34375, 54.5625, 17.84375...","[30.609375, 29.40625, 60.03125, 52.375, 14.367...","[29.9375, 28.390625, 59.5, 50.5625, 13.140625,...","[28.96875, 27.453125, 58.96875, 49.46875, 11.7...","[27.1875, 26.46875, 58.53125, 48.59375, 11.203...","[24.875, 25.453125, 58.125, 48.0, 10.9609375, ...",...,1790.729475,True,True,False,True,True,True,True,True,False
4,9,"[51.4375, 27.515625, 51.4375, 51.4375, 18.6875...","[48.59375, 26.671875, 48.59375, 48.59375, 15.0...","[56.65625, 29.5625, 56.65625, 56.65625, 27.312...","[46.03125, 25.9375, 46.03125, 46.03125, 13.0, ...","[43.53125, 25.296875, 43.53125, 43.53125, 11.3...","[41.28125, 24.671875, 41.28125, 41.28125, 10.3...","[39.15625, 24.046875, 39.15625, 39.15625, 9.82...","[37.0, 23.359375, 37.0, 37.0, 9.4609375, 11.28...","[35.125, 22.625, 35.125, 35.125, 9.109375, 9.3...",...,1208.705149,True,False,False,False,False,False,False,False,False


In [4]:
#see AppApi.add_confunder_dose_limits for the <Organ>_limit at the end
#based on rules in literature cited in the paper
for col in data.columns:
    #skip similar values for the symptoms ect
    if 'symptoms' in col and 'drymouth' not in col:
        continue
    if len(col) == 3 and col[0] in ['V','D']:
        continue
    print(col,data[col].iloc[0])
    print()

id 2

D2 [54.53125, 28.1875, 59.84375, 57.40625, 55.9375, 63.71875, 56.125, 73.1875, 26.53125, 70.375, 71.0625, 60.875, 55.90625, 56.125, 63.46875, 43.90625, 36.6875, 72.5625, 58.84375, 67.8125, 72.375, 73.0, 61.5, 60.875, 73.0625, 72.875, 64.9375, 72.5625, 73.1875, 72.375, 72.625, 58.90625, 72.375, 72.1875, 72.5625, 71.875, 52.875, 35.5625, 20.453125, 48.5625]

D5 [53.21875, 27.4375, 58.28125, 55.5, 54.90625, 60.875, 53.3125, 73.0, 24.875, 62.65625, 69.875, 60.5, 55.71875, 52.53125, 60.84375, 38.21875, 34.8125, 72.125, 57.71875, 59.6875, 72.25, 72.625, 55.84375, 58.25, 72.5625, 72.4375, 63.90625, 72.3125, 72.8125, 72.1875, 72.3125, 51.375, 71.8125, 71.875, 72.25, 71.1875, 41.125, 32.375, 17.234375, 44.96875]

V5 [96.375, 100.0, 95.6875, 100.0, 100.0, 100.0, 100.0, 100.0, 30.71875, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 93.6875, 97.375, 100.0, 100.0, 100.0, 100.0, 100.0, 91.9375, 97.875, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 99.6875, 87.75, 100

In [5]:
Const.organ_list

['Esophagus',
 'Spinal_Cord',
 'Lt_Brachial_Plexus',
 'Rt_Brachial_Plexus',
 'Cricopharyngeal_Muscle',
 'Cricoid_cartilage',
 'IPC',
 'MPC',
 'Brainstem',
 'Larynx',
 'Thyroid_cartilage',
 'Rt_Sternocleidomastoid_M',
 'Rt_Mastoid',
 'Rt_Parotid_Gland',
 'Rt_Medial_Pterygoid_M',
 'Rt_Lateral_Pterygoid_M',
 'Rt_Masseter_M',
 'Lt_Sternocleidomastoid_M',
 'Lt_Mastoid',
 'Lt_Parotid_Gland',
 'Lt_Submandibular_Gland',
 'Lt_Medial_Pterygoid_M',
 'Lt_Lateral_Pterygoid_M',
 'Lt_Masseter_M',
 'Supraglottic_Larynx',
 'SPC',
 'Rt_Submandibular_Gland',
 'Hyoid_bone',
 'Soft_Palate',
 'Genioglossus_M',
 'Tongue',
 'Rt_Ant_Digastric_M',
 'Lt_Ant_Digastric_M',
 'Mylogeniohyoid_M',
 'Extended_Oral_Cavity',
 'Mandible',
 'Hard_Palate',
 'Lower_Lip',
 'Upper_Lip',
 'Glottic_Area']

In [31]:
#clustering works conceptuallby be selecting a set of dose/organ features and a set of organs,
#and converting that into a 1d vector for each patient ,which is input into cluster
def to_value_array(df,features,organ_subset):
    organ_positions = [Const.organ_list.index(o) for o in organ_subset]
    vals = np.stack(df[features].apply(lambda x: np.stack([np.array([ii[i] for i in organ_positions]).astype(float) for ii in x]).ravel(),axis=1).values)
    return vals

to_value_array(data,features=['V15','V25'],organ_subset=['IPC','MPC','SPC'])

array([[100.     , 100.     , 100.     , 100.     , 100.     , 100.     ],
       [100.     , 100.     , 100.     , 100.     , 100.     , 100.     ],
       [100.     , 100.     , 100.     , 100.     , 100.     , 100.     ],
       ...,
       [ 17.125  ,  80.     , 100.     ,   0.     ,  39.40625,  92.25   ],
       [100.     , 100.     , 100.     , 100.     , 100.     , 100.     ],
       [ 73.25   , 100.     ,  99.5    ,  55.5    ,  99.625  ,  93.     ]])

In [32]:
#clustering is done in the add_sd_dose_clusters in appAPi
#this does the function of to_value_array and then runs clustering on the data
#by default, we use a gaussian mixture model
#this returns a dataframe with a 'dose_clusters' column with the cluster labels
clusters = add_sd_dose_clusters(data,
                     features=['V15','V25'], #dose features e.g. VX, DX, mean_dose, volume
                     organ_subset=['IPC','MPC','SPC'], #organs to use in clustering
                     n_clusters=7, #number of clusters
                     normalize=True #whether to normalize the input data
                    )
clusters[['id','dose_clusters']].iloc[0:10]

,id,dose_clusters
0,2,6
1,6,6
2,7,6
3,8,6
4,9,1
5,10,6
6,11,6
7,12,1
8,13,5
9,15,5


In [26]:
#clusters are ordered by mean_dose in reorder_clusters column (change the "by" argument to reorder clusters to change)
#this orders by the average number of people with t3 or t4
reorder_clusters(clusters,'dose_clusters',by='t_severe')[['id','dose_clusters']].iloc[0:10]

,id,dose_clusters
0,2,6
1,6,6
2,7,6
3,8,6
4,9,3
5,10,6
6,11,6
7,12,3
8,13,4
9,15,4


In [39]:
#We can also add in custom clustering and reducers
#reducers are e.g. dimensionality reduction done before the clustering
#reduers may also work with an autoencoder if you add a wrapper function to work with reducer.fit_transform(numpy_array)
from sklearn.decomposition import PCA

#see keyword_clusterer to see predefned cluster options
add_sd_dose_clusters(data,
                     clusterer=keyword_clusterer('kmeans',6),
                     reducer=PCA(3),
                     features=['V15','V25'], #dose features e.g. VX, DX, mean_dose, volume
                     organ_subset=['IPC','MPC','SPC'], #organs to use in clustering
                     normalize=True #whether to normalize the input data
                    )[['id','dose_clusters']].iloc[0:10]

,id,dose_clusters
0,2,5
1,6,5
2,7,5
3,8,5
4,9,2
5,10,5
6,11,5
7,12,1
8,13,4
9,15,4


In [49]:
#we can preform likelihood ratio tests with the clusters and confounders for different binary symptom outcomes.
#this returns the dataframe with columns with results in the form 'cluster_<symptom>_<threshold>_<timepoint>' for each option tested
lrt_results = get_cluster_lrt(clusters,
                              symptoms=['drymouth'], #symptoms to test
                              confounders=['t4','n3'], #confounders to use in the lrt test
                              nWeekList=[[13,33]], #list of sets of week. we take the max over each list, so this is max drymouth between 6 weeks and 6 months post treatment
                              thresholds=[5]) #severity thresholds totest
lrt_results[['dose_clusters']+[c for c in lrt_results if 'cluster_drymouth_5_33wks' in c]].groupby('dose_clusters').first()

/media/andrew/TheBigOne/git_repos/QubbedDataAnalysis/python/../backend/AppApi.py:301: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6236112158343292' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  if not pd.isnull(res[name]):
/media/andrew/TheBigOne/git_repos/QubbedDataAnalysis/python/../backend/AppApi.py:301: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.4905023845967025' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  if not pd.isnull(res[name]):
/media/andrew/TheBigOne/git_repos/QubbedDataAnalysis/python/../backend/AppApi.py:301: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6237784428324346' has dtype incompatible with int64, please explicitly cast t

,cluster_drymouth_5_33wks_lrt_pval,cluster_drymouth_5_33wks_ttest_tval,cluster_drymouth_5_33wks_ttest_pval,cluster_drymouth_5_33wks_aic_diff,cluster_drymouth_5_33wks_odds_ratio
dose_clusters,,,,,
0,0.000022,-3.485202,0.000492,-16.028362,0.153821
1,0.000018,-3.809431,0.000139,-16.427846,0.243249
2,0.000236,-3.312148,0.000926,-11.519969,0.268069
3,0.017113,-2.217772,0.026570,-3.684748,0.351863
4,0.146260,-1.413318,0.157562,-0.110835,0.518717
5,0.010895,-2.499069,0.012452,-4.482440,0.596503
6,0.002799,-2.953388,0.003143,-6.933754,0.620325


In [ ]:
#This section deals with rule explainers

In [63]:
#this processes the data into inputs in terms of dose featuers and outcomes
temp_x, temp_y = get_rule_inference_data(clusters,
                                         ['IPC','SPC'], #organs for inputs to consider
                                         ['drymouth'], #outcomes to consider
                                         ['V25','V30','mean_dose'],
                                         [13,33],
                                         include_limits=True)
temp_x.head()

,V25_IPC,V25_SPC,V30_IPC,V30_SPC,mean_dose_IPC,mean_dose_SPC,Parotid_Gland_limit,IPC_limit,IPC_limit2,MPC_limit,MPC_limit2,SPC_limit,SPC_limit2,Larynx_limit,Esophagus_limit
0,100.000000,100.0,99.125000,100.00000,41.851960,68.475769,1,0,0,1,1,1,1,0,1
1,100.000000,100.0,100.000000,100.00000,43.879899,66.566609,1,0,0,1,0,1,1,0,1
2,100.000000,100.0,100.000000,100.00000,50.744640,59.884944,1,1,0,1,0,1,0,0,0
3,100.000000,100.0,100.000000,100.00000,52.224735,67.431397,1,1,0,1,1,1,1,1,0
4,21.078125,72.0,16.765625,61.28125,17.433103,43.319443,1,0,0,0,0,0,0,0,0


In [62]:
temp_y.head()

,drymouth
0,8.0
1,8.0
2,8.0
3,5.0
4,3.0


In [73]:
#this generates a set of rules. second input is assumed to be a binary dataframe, but can technically have any integer input
#here we're generating rules to predict if a patient has drymouth (this is an option in the interface)
temprules = get_rule_df(temp_x,pd.DataFrame(temp_y > 5))
for k,v in temprules[0].items():
    print(k,v)
    print()

features ['V25_IPC']

thresholds [np.float64(30.0)]

splits [0       True
1       True
2       True
3       True
4      False
       ...  
479    False
480    False
481    False
482     True
483     True
Name: V25_IPC, Length: 484, dtype: bool]

rule 0       True
1       True
2       True
3       True
4      False
       ...  
479    False
480    False
481    False
482     True
483     True
Name: V25_IPC, Length: 484, dtype: bool

info 0.01025155000751568

odds_ratio 1.4857142857142855

lower_count 99

lower_tp 36

lower_mean 0.36363636363636365

upper_count 385

upper_tp 208

upper_mean 0.5402597402597402



In [92]:
#for generating rule explanations we actually use cluster membership 
#this tests all splits and save the memories and stuff lol
cluster_outcome = clusters.dose_clusters.apply(lambda x: x == clusters.dose_clusters.max())
cluster_rules = get_rule_df(temp_x,pd.DataFrame(cluster_outcome))
[[k['features'],k['thresholds'],k['info']] for k in cluster_rules[0:4]]

[[['V25_IPC'], [np.float64(2.0)], np.float64(0.04501218067514473)],
 [['V25_IPC'], [np.float64(4.0)], np.float64(0.050184180770673936)],
 [['V25_IPC'], [np.float64(6.0)], np.float64(0.05148734211258693)],
 [['V25_IPC'], [np.float64(8.0)], np.float64(0.06206316488772695)]]

In [102]:
#This is a version of the AppApi call that gets the rules to return to the interface, in case you want to generate those,
#The difference here is that in the AppAPI the cluster labels and arguments are passed as a json file in the request

#this returns of list of dictionaries with information about the rules
def get_rules(df,
                   organs=Const.organ_list[:],#organs to check
                   symptoms=['drymouth'],#symptom to check. I forget why it's a list and not just one
                   organ_features=['V5','V10','V15','V20','V25','V30','V35','V40','V45','V50','V55','V60','V65','V70','V75','V80','mean_dose','max_dose'],
                   s_dates=[33],
                   threshold=5, #threshold for the rule. e.g. we want to predict symptoms[0] > threshold
                   maxdepth=3,#max number of thresholds
                   min_odds=0,#used to trim the forward search
                   min_info=.08,#used to trim the forward search
                   criteria='info',#criteria used to chose rules, info or odds_ratio. odds_ratio is only positive
                   max_rules=3,#max number of rule sets to return
                   max_frontier=10,#used to trim forward search. 
                   granularity=1,#how granular to go in the splits (e.g. every 1 rad, every .1 rads
                   predict_cluster = -1,#if 1 or true, predict the cluster membership. otherwise we use the symptoms
                   cluster=None,
                   use_limits=False#wether to use dose limits in addition to the other stuff idk
             ):
    df.post_cluster = df.dose_clusters
    df = df.set_index('id')
    if predict_cluster is not None and predict_cluster >= 0:
        cluster = None
    dose_df, outcome_df = get_rule_inference_data(
            df,
            organs,
            symptoms,
            organ_features,
            s_dates,
            cluster=cluster,
            include_limits=use_limits,
        )
    symptom_bool = (outcome_df>=threshold)
    if predict_cluster is not None and predict_cluster >= 0:
        df['temp_outcome'] = df.post_cluster.apply(lambda x: x == predict_cluster)
        y = df[['temp_outcome']]
    else:
        y = symptom_bool
    rules = get_rule_df(dose_df,y,min_odds=min_odds,granularity=granularity)
    sort_rules = lambda rlist: sorted(rlist, key=lambda x: -x[criteria])
    rules = sort_rules(rules)
    min_info = min(rules[0].get('info',0.0)*.6,float(min_info))
    rules = [r for r in rules if r.get('info',0) >= min_info]
    if len(rules) > 800:
        rules = rules[:800]
    print('n rules',len(rules))
    frontier = [None]
    best_rules = []
    depth = 0
    while (depth < maxdepth) and (frontier is not None) and (len(frontier) > 0):
        frontier = get_best_rules(frontier,rules,y,min_odds=min_odds,criteria=criteria)
        frontier = sorted(frontier, key = lambda x: -x[criteria] if x is not None else 0)
        frontier = frontier[:max_frontier]
        depth += 1
        best_rules.extend(frontier)
        print('lb',len(best_rules))
        print()
    
    best_rules = sort_rules(best_rules)
    best_rules = best_rules[:max_rules]
    best_rules = joblib.Parallel(n_jobs=-2)(joblib.delayed(format_rule_json)((br,y,symptom_bool)) for br in best_rules)
    print([(r['features'],r.get('info')) for r in best_rules])
    pos_ids = y[y.values.astype(bool)].index.values.tolist()    
    for br in best_rules:
        br['target_ids'] = pos_ids
    return best_rules
    
test_rules = get_rules(clusters)
print(len(test_rules),'rules generated')
best_rule = test_rules[0]
print('best rule for predicting drymouth > 5 of the selected organs')
print('with a mutual information score with target outcome of ',best_rule['info'])
for organ,threshold in zip(best_rule['features'],best_rule['thresholds']):
    print(organ + ' > ' + str(threshold))


3 rules generated
best rule for predicting drymouth > 5 of the selected organs
with a mutual information score with target outcome of  0.04798877998068714
V25_Brainstem > 13.0
V15_Rt_Parotid_Gland > 41.0
V55_Rt_Submandibular_Gland > 7.0


In [103]:
#full breakdown of returned items
#upper and lower ids are the patient ids of the people above and below the threshold
#target ids is the list of people with a 1 in the thing we're predicting
for k,v in test_rules[0].items():
    print(k)
    print(v)
    print()

features
['V25_Brainstem', 'V15_Rt_Parotid_Gland', 'V55_Rt_Submandibular_Gland']

thresholds
[np.float64(13.0), np.float64(41.0), np.float64(7.0)]

info
0.04798877998068714

odds_ratio
2.3114754098360657

lower_count
423

lower_tp
147

lower_mean
0.3475177304964539

upper_count
61

upper_tp
49

upper_mean
0.8032786885245902

upper_ids
[7, 8, 15, 24, 43, 48, 49, 52, 61, 71, 73, 86, 93, 101, 102, 127, 139, 152, 161, 163, 186, 219, 221, 225, 235, 236, 247, 248, 258, 293, 323, 343, 362, 367, 368, 369, 403, 411, 417, 421, 425, 429, 475, 509, 514, 520, 534, 564, 585, 604, 609, 612, 614, 615, 616, 648, 680, 685, 692, 723, 724]

lower_ids
[2, 6, 9, 10, 11, 12, 13, 16, 18, 19, 21, 22, 23, 25, 28, 29, 31, 32, 34, 35, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 50, 51, 53, 54, 56, 57, 58, 59, 60, 62, 65, 67, 68, 69, 72, 76, 77, 78, 79, 80, 81, 82, 84, 85, 89, 91, 92, 94, 95, 96, 97, 98, 99, 100, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 116, 117, 120, 121, 123, 126, 128, 132, 133, 134, 135, 1